In [4]:
import os, json, glob, random
from pathlib import Path

# 設定１のjsonファイルの作成

設定1の訓練の場合 \
ベースタスクに異常データを混入させ，追加タスクは正常のみを対象とする． \

設定1の検証の場合 \
ベースタスクの検証は，訓練に追加した異常を検証対象から外す． \
追加タスクの検証では，ベースタスクに混入した異常タイプを異常としてテストする． \
訓練に追加した "xxx.png" に関する内容はjsonから手動で削除する必要があります． \

追加タスクは元のjsonファイルをそのまま使用する．


In [ ]:
"""
訓練用 json ファイルの作成
"""

# 設定
setting = 1    # ベースタスク：14クラス，追加タスク：1クラス * 1タスク

# ベースパス
base_path = "/home/kouyou/ContinualLearning/repexp/One-for-More/data/mvtec_anomaly_detection_for_mu"

# 訓練に異常を混ぜるカテゴリ
ano_category = ["bottle", "metal_nut", "tile", "screw"]

# 異常のタイプ
ano_type = ["broken_small", "color", "gray_stroke", "scratch_neck"]

# 使う回転（保存済みがあるときのみ参照）
angles = [0, 90, 180, 270]

# 何枚混ぜるか（カテゴリごと）
k_per_class = 20


# JSON の場所（One-for-More の既定パスに合わせる）
json_dir = f"./data/MVTec-AD-MU/{setting}"
json_orig_dir = f"./data/MVTec-AD/{setting}"
in_train_json  = f"{json_orig_dir}/train_1.json"   # 追記元（ベースタスク）
out_train_json = f"{json_dir}/train_1.json"        # 出力先

# 既存の train_1 を読み込み
with open(in_train_json, "rt") as f:
    train_items = json.load(f)


new_items = []

for cls, defect in zip(ano_category, ano_type):

    # 作成した訓練用異常データ（扱いとしては正常）が保存しているディレクトリ
    path_dir = os.path.join(base_path, cls, "train", defect, "rot_aug")
    print(path_dir)

    # path_dir に含まれる画像を in_train_json に追加
    files = sorted(glob.glob(os.path.join(path_dir, "*.png")))
    if not files:
        print(f"[warn] no images under {path_dir}")
        continue


    # 最大 k_per_class 枚だけ追加
    picked = random.sample(files, k=min(k_per_class, len(files)))
    for p in picked:
        rel = str(Path(p).relative_to(base_path))  # JSON 用の相対パス
        new_items.append({
            "filename": rel,
            "label": 0,                 # ← 正常として混入
            "label_name": "good",
            "clsname": cls,
            "contaminated": True        # 追跡用メタ（任意）
        })


# 既存と結合して出力
os.makedirs(json_dir, exist_ok=True)
aug_items = train_items + new_items
with open(out_train_json, "wt") as f:
    json.dump(aug_items, f, indent=2, ensure_ascii=False)

print(f"[done] wrote {out_train_json} (+{len(new_items)} items, total {len(aug_items)})")

/home/kouyou/ContinualLearning/repexp/One-for-More/data/mvtec_anomaly_detection_for_mu/bottle/train/broken_small/rot_aug
/home/kouyou/ContinualLearning/repexp/One-for-More/data/mvtec_anomaly_detection_for_mu/metal_nut/train/color/rot_aug
/home/kouyou/ContinualLearning/repexp/One-for-More/data/mvtec_anomaly_detection_for_mu/tile/train/gray_stroke/rot_aug
/home/kouyou/ContinualLearning/repexp/One-for-More/data/mvtec_anomaly_detection_for_mu/screw/train/scratch_neck/rot_aug
[done] wrote ./data/MVTec-AD-MU/1/train_1.json (+80 items, total 3469)


In [15]:
"""
検証用 json ファイルの作成
"""

# 訓練に異常を混ぜるカテゴリ
ano_category = ["bottle", "metal_nut", "tile", "screw"]

# 異常のタイプ
ano_type = ["broken_small", "color", "gray_stroke", "scratch_neck"]

# JSON の場所（One-for-More の既定パスに合わせる）
json_dir = f"./data/MVTec-AD-MU/{setting}"
json_orig_dir = f"./data/MVTec-AD/{setting}"
in_train_json  = f"{json_orig_dir}/test_1.json"   # 追記元（ベースタスク）
out_train_json = f"{json_dir}/test_1.json"        # 出力先

# 入力JSONを読み込み
with open(in_train_json, "rt") as f:
    items = json.load(f)

# cls → defect の対応を作成
target_map = {c: d for c, d in zip(ano_category, ano_type)}



def extract_defect_type(relpath: str) -> str | None:
    """
    'bottle/test/broken_small/005.png' → 'broken_small'
    'bottle/test/good/005.png'         → 'good'
    形式外は None を返す
    """
    parts = Path(relpath).parts
    if "test" not in parts:
        return None
    i = parts.index("test")
    return parts[i + 1] if i + 1 < len(parts) else None


# 指定の (cls, defect) に一致する test サンプルを「正常ラベル」に変更
changed = 0
for it in items:
    cls = it.get("clsname")
    if cls not in target_map:
        continue
    defect = extract_defect_type(it.get("filename", ""))
    if defect is None or defect == "good":
        continue
    if defect == target_map[cls]:
        it["label"] = 0
        it["label_name"] = "good"
        it.pop("maskname", None)   # 正常扱いなのでピクセルGTは削除
        changed += 1

# 出力
with open(out_train_json, "wt") as f:
    json.dump(items, f, indent=2, ensure_ascii=False)

print(f"[done] wrote {out_train_json} (relabelled {changed} items)")

[done] wrote ./data/MVTec-AD-MU/1/test_1.json (relabelled 85 items)
